<a href="https://colab.research.google.com/github/DTAMBU/03MAIR_Algoritmos_de_Optimizacion_2023_2024/blob/main/Damian_Tamburi_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Actividad Guiada 3 de Algoritmos de Optimizacion

Nombre: Damian Tamburi

https://colab.research.google.com/drive/1RVxueryluA3YxYXzBX-ee-Y3iGuQHnCG

https://github.com/DTAMBU/03MAIR_Algoritmos_de_Optimizacion_2023_2024/blob/main/Damian_Tamburi_AG3.ipynb

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios


#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? ^C


In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [ ]:
#Probamos algunas funciones del objeto problema

#Distancia entre nodos
problem.get_weight(0, 1)

15

#Funcionas basicas


In [ ]:
#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

sol_temporal = crear_solucion(Nodos)

distancia_total(sol_temporal, problem), sol_temporal

(4869,
 [0,
  19,
  31,
  4,
  20,
  15,
  6,
  16,
  32,
  34,
  14,
  13,
  24,
  27,
  36,
  22,
  38,
  35,
  28,
  1,
  18,
  2,
  23,
  9,
  30,
  12,
  7,
  41,
  8,
  17,
  26,
  33,
  10,
  11,
  21,
  3,
  39,
  29,
  25,
  37,
  40,
  5])

#BUSQUEDA ALEATORIA

In [ ]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 10000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 1, 7, 22, 26, 20, 17, 32, 13, 37, 19, 12, 4, 23, 15, 40, 21, 9, 8, 30, 16, 34, 33, 38, 39, 10, 24, 41, 11, 25, 29, 31, 14, 28, 5, 36, 35, 27, 3, 2, 6, 18]
Distancia     : 3766


#BUSQUEDA LOCAL

In [ ]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3766
Distancia Mejor Solucion Local: 3328


In [ ]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

En la iteracion  31 , la mejor solución encontrada es: [0, 1, 5, 7, 31, 32, 34, 33, 20, 35, 36, 17, 37, 15, 16, 14, 19, 13, 26, 8, 40, 24, 21, 39, 2, 4, 6, 3, 27, 28, 29, 30, 38, 22, 9, 23, 41, 10, 25, 11, 12, 18]
Distancia     : 1658


Entorno de vecindad variable:

la variable, "n" se incrementa si no se encuentra una mejor solución y se disminuye si se encuentra una mejor solución. La idea es que si estamos encontrando buenas soluciones, entonces estamos en una buena región del espacio de búsqueda y podemos permitirnos explorar un entorno de vecindad más pequeño. Por otro lado, si no estamos encontrando buenas soluciones, entonces podríamos estar atrapados en un mínimo local y necesitamos explorar un entorno de vecindad más grande para intentar escapar.

In [ ]:
def genera_vecina_2(solucion, n):
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-n):
    for j in range(i+n, len(solucion)):

      vecina = solucion[:i] + solucion[j:j+n] + solucion[i+n:j] + solucion[i:i+n] + solucion[j+n:]

      distancia_vecina = distancia_total(vecina, problem)

      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

def busqueda_local_adaptativa(problem, N):
    Nodos = list(problem.get_nodes())
    mejor_solucion = []
    mejor_distancia = float('inf')

    #Generar una solucion inicial de referencia(aleatoria)
    solucion_referencia = crear_solucion(Nodos)
    mejor_distancia = distancia_total(solucion_referencia, problem)

    # Inicializamos el tamaño del entorno de vecindad
    n = 2

    iteracion=0             #Un contador para saber las iteraciones que hacemos
    while(1):
        iteracion +=1         #Incrementamos el contador

        #Obtenemos la mejor vecina ...
        vecina = genera_vecina_2(solucion_referencia, n)

        #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
        distancia_vecina = distancia_total(vecina, problem)

        #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina                   #Guarda la mejor solución encontrada
            mejor_distancia = distancia_vecina
            n = max(2, n - 1)  # Si encontramos una mejor solución, disminuimos n (pero nunca menos de 2)
        else:
            n = min(len(Nodos) - 1, n + 1)  # Si no encontramos una mejor solución, aumentamos n (pero nunca más que el número de nodos - 1)

        solucion_referencia = vecina

        # Condición de parada: si n alcanza su valor máximo, terminamos
        if n == len(Nodos) - 1:
            print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
            print("Distancia     :" , mejor_distancia)
            return mejor_solucion

In [ ]:
sol2 = busqueda_local_adaptativa(problem, 1000)

En la iteracion  79 , la mejor solución encontrada es: [0, 27, 3, 4, 5, 26, 18, 11, 10, 41, 23, 21, 40, 24, 39, 22, 38, 34, 20, 33, 32, 30, 28, 2, 29, 8, 9, 25, 12, 13, 19, 6, 1, 7, 14, 16, 15, 37, 17, 36, 35, 31]
Distancia     : 1584


#SIMULATED ANNEALING


In [ ]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))

  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]


#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

In [ ]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo


  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 26, 5, 13, 7, 31, 1, 6, 16, 15, 14, 18, 12, 10, 28, 30, 32, 36, 35, 20, 33, 39, 24, 40, 21, 9, 23, 41, 8, 4, 3, 2, 27, 34, 17, 37, 19, 11, 25, 29, 22, 38]
con una distancia total de 2241


SIMULATED ANNEALING con busqueda local:

In [ ]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))

  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]


#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

In [ ]:
def recocido_simulado(problem, TEMPERATURA ):
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = 10e100

  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    vecina = genera_vecina(solucion_referencia)  # Modificado aquí

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 3, 1, 7, 37, 17, 36, 35, 31, 32, 30, 21, 24, 40, 41, 10, 25, 11, 12, 18, 26, 4, 6, 5, 13, 19, 16, 15, 14, 2, 27, 28, 38, 22, 39, 9, 23, 8, 29, 34, 33, 20]
con una distancia total de 1720


In [ ]:
def recocido_simulado(problem, TEMPERATURA ):
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = 10e100

  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera una solución vecina
    # Inicializamos el tamaño del entorno de vecindad
    n = 2
    vecina = genera_vecina_2(solucion_referencia,n)  # Modificado aquí

    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 3, 2, 30, 28, 32, 34, 35, 36, 37, 14, 5, 19, 13, 16, 15, 17, 31, 20, 33, 38, 22, 40, 24, 21, 39, 29, 9, 8, 41, 23, 25, 12, 11, 10, 27, 26, 18, 6, 4, 1, 7]
con una distancia total de 1805
